In [2]:
import sys
sys.path.append("/home/scorpio/ngsild-pythonclient-initial_ngsild_client/src/")

import ast
import json
from ngsildclient.client import Client
from ngsildclient.entity import Entity
import csv
import time
from datetime import datetime
import pandas as pd 
import numpy as np
from geojson.geometry import Point, Polygon, MultiLineString

client_obj = Client("localhost", "9090") # connect to scorpio

def publish(entity):
    response = client_obj.temporal_create(entity)
    #print(response)
    if response['status_code']> 400:
        print("ERROR: Cannot publish data from the NGSI client.")
    return response



In [3]:
num_rows_batch_publish=5000 # number of objects to be batch published to Scorpio

# Notes:
Consider every hexagon as an entity, and sensor data as properties

In [4]:
import pandas as pd
import geopandas as gpd

from geojson.geometry import Point, Polygon, MultiLineString


In [5]:
gdf_NO2 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_NO2.pkl')
gdf_O3 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_O3.pkl')
gdf_PM10 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_PM10.pkl')
gdf_PM2_5 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_PM2.5.pkl')
gdf_SO2 = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_SO2.pkl')
gdf_CO = pd.read_pickle('./../cgarrido/madrid_data/air_pollution/gdf_aggr_CO.pkl')



In [ ]:
pickle.dump(your_dictionary, open('pickle_file_name.p', 'wb'))


In [6]:
gdf_SO2 = pd.read_pickle('./../zxu//gdf_aggr_SO2.pkl')


,hexagon_id,hexagon_x,hexagon_y,datetime,year,month,day,hour,minute,weekday,data,lat,lon,altitude,geometry
0,36,3,3,2022-01-01 00:00:00,2022,1,1,0,0,5,31.0,40.412544,-3.758228,646.0,"POLYGON ((-418731.513 4927969.963, -420105.448..."
1,5,1,0,2022-01-01 00:00:00,2022,1,1,0,0,5,18.0,40.442185,-3.693213,680.0,"POLYGON ((-411492.801 4932304.955, -412867.358..."
2,4,0,1,2022-01-01 00:00:00,2022,1,1,0,0,5,20.0,40.403298,-3.683349,600.0,"POLYGON ((-410394.293 4926617.691, -411768.130..."
3,29,0,-3,2022-01-01 00:00:00,2022,1,1,0,0,5,16.0,40.489185,-3.647097,700.0,"POLYGON ((-406358.341 4939183.058, -407733.827..."
4,26,1,3,2022-01-01 00:00:00,2022,1,1,0,0,5,28.0,40.377738,-3.720362,605.0,"POLYGON ((-414515.288 4922881.427, -415888.609..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210235,26,1,3,2022-12-31 23:45:00,2022,12,31,23,45,5,25.0,40.377738,-3.720362,605.0,"POLYGON ((-414515.288 4922881.427, -415888.609..."
210236,29,0,-3,2022-12-31 23:45:00,2022,12,31,23,45,5,20.0,40.489185,-3.647097,700.0,"POLYGON ((-406358.341 4939183.058, -407733.827..."
210237,36,3,3,2022-12-31 23:45:00,2022,12,31,23,45,5,23.0,40.412544,-3.758228,646.0,"POLYGON ((-418731.513 4927969.963, -420105.448..."
210238,4,0,1,2022-12-31 23:45:00,2022,12,31,23,45,5,24.0,40.403298,-3.683349,600.0,"POLYGON ((-410394.293 4926617.691, -411768.130..."


In [9]:

gdf_list= [gdf_PM2_5,  gdf_NO2, gdf_O3, gdf_PM10, gdf_SO2, gdf_CO]
gdf_property_list=["PM2_5", "NO2", "O3", "PM10",  "SO2", "CO"]
entity_id_prefix =  "urn:ngsi-ld:AirQualityObserved:Madrid"
entity_type = 'AirQualityObserved'


In [1]:
gdf_PM2_5

NameError: name 'gdf_PM2_5' is not defined

1- Check context and add context (smart data model link) to the @context tag

2- Geometry: Change geometry to location. Location is a geoproperty. Give the point values as "coordinates". Keep the observedat on every property.

3- Convert coordinates to lon/lat

4- Add no2, so2 as property (not as a separate entity). One entity per hexagon
. 
5- Add district instead of hexagons


In [11]:

example_flag=False
for property_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
    ingestion_starttime= time.perf_counter() 
    example_flag=True
    prop =gdf_property_list[property_index]

    hexagon_id_list = tmp_gdf['hexagon_id'].unique()
    print("Info: Unique hexagons for the dataset : "+ str(hexagon_id_list))
    feature_list= tmp_gdf.columns.tolist()
    dataset_length = len(tmp_gdf)
    for hexagon_id in hexagon_id_list:
        hexagon_gdf = tmp_gdf[tmp_gdf['hexagon_id']== hexagon_id]
        
        entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity

        counter =0
        for index, row in hexagon_gdf.iterrows():
            lat=0
            lon=0
            if counter %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
                entity_obj = Entity(entity_type, entity_id_prefix + ':Hexagon:' + str(hexagon_id)) # (re-)initialize the entity
            for f in range(len(feature_list)):
                if feature_list[f] =='type': feature_list[f] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
                if feature_list[f] == 'lat': lat = row[f]
                if feature_list[f] == 'lon': lon=row[f]
                if lat !=0 and lon !=0: # both values are obtained for location, add the location attribute 
                    location_value = Point((lat, lon))
                    entity_obj.add_geoprop("location", location_value, observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S')) # MAKE THIS A POINT (SIVA EXAMPLE)
                elif feature_list[f] == 'data':
                    entity_obj.add_prop(prop, str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
                elif(feature_list[f] not in ['datetime','type']):
                    entity_obj.add_prop(feature_list[f], str(row[f]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
            if example_flag:
                print('INFO: Example data from the dataset to be published - as an example on how data looks.')
                print(entity_obj)
                example_flag=False

            if (counter %num_rows_batch_publish==0 or counter == len(hexagon_gdf)-1) and  counter!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
                publish(entity_obj.to_dict()) # publish the entity
            counter+=1 
        print("... published data for hexagon: " +  str(hexagon_id))

    ingestion_endtime= time.perf_counter() 
    print('Info: Dataset is published successfully' 
        
          + ' --- It took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds to publish the dataset with length ' + str(dataset_length))


Info: Unique hexagons for the dataset : [36  5  4 29 26 14]
INFO: Example data from the dataset to be published - as an example on how data looks.
{'id': 'urn:ngsi-ld:AirQualityObserved:Madrid:Hexagon:36', 'type': 'AirQualityObserved', '@context': ['https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld'], 'hexagon_id': {'type': 'Property', 'value': '36', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_x': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'hexagon_y': {'type': 'Property', 'value': '3', 'observedAt': '2022-01-01T00:00:00Z'}, 'year': {'type': 'Property', 'value': '2022', 'observedAt': '2022-01-01T00:00:00Z'}, 'month': {'type': 'Property', 'value': '1', 'observedAt': '2022-01-01T00:00:00Z'}, 'day': {'type': 'Property', 'value': '1', 'observedAt': '2022-01-01T00:00:00Z'}, 'hour': {'type': 'Property', 'value': '0', 'observedAt': '2022-01-01T00:00:00Z'}, 'minute': {'type': 'Property', 'value': '0', 'observedAt': '2022-01-01T00:00:00Z'}, 'weekday'

# Query temporal entity from Scorpio and get it as Pandas

In [ ]:

client_obj = Client("localhost", "9090")
pandas_data= client_obj.get_temporal("urn:ngsi-ld:AirQualityObserved:Madrid:Hexagon:0", entityType="AirQualityObserved", pandasSeries=True, attribute="value")
print(pandas_data)
print(type(pandas_data))



## BACKUP CODE

import time


for entity_index, tmp_gdf in enumerate(gdf_list): # for each dataset 
        
    feature_list= tmp_gdf.columns.tolist()
    for index, row in tmp_gdf.iterrows():
        if index %num_rows_batch_publish==0: # create a new clean entity everytime a batch is completed/published to avoid duplicate push of same values 
            entity_obj = Entity(entity_type_list[entity_index], entity_id_list[entity_index]) # (re-)initialize the entity
        for i in range(len(feature_list)):
            if feature_list[i] =='type': feature_list[i] ='road_type' # quick fix for the error of "type" conflicting with ngsi-ld "type"
            if(feature_list[i] not in ['datetime','type']):
                entity_obj.add_prop(feature_list[i], str(row[i]) , observedAt= datetime.strptime(str(row.datetime), '%Y-%m-%d %H:%M:%S'))                 
        if index == 0:
            print('INFO: Initial row of the dataset to be published - as an example on how data looks.')
            print(entity_obj)
            
        #ingestion_starttime= time.perf_counter()
        temp_objects.append(entity_obj)
        if (index %num_rows_batch_publish==0 or index == len(tmp_gdf)-1) and  index!=0: # publish the batch data for entity to scorpio as the rows fulfill the number of rows needed for a batch, or the dataset has come to the end
            publish(entity_obj.to_dict()) # publish the entity
    
        f(index%10000 ==0):
            print("... published data count: " +  str(index))

        #print('Info: Ingestion took ' + str(ingestion_endtime -ingestion_starttime) + ' seconds for ' +  str(num_rows_batch_publish) + ' objects')

    print('Info: Dataset is published successfully with entity ID: ' + str(entity_id_list[entity_index]))
